# Día 3: Principios DRY y KISS

## Descripción General

Los principios DRY (Don't Repeat Yourself) y KISS (Keep It Simple, Stupid) son dos de los fundamentos más importantes del código limpio. DRY nos enseña a evitar la duplicación, mientras que KISS nos recuerda mantener las soluciones simples. Juntos, estos principios nos guían hacia código mantenible, comprensible y robusto.

En este notebook aprenderás cómo aplicar ambos principios de manera efectiva y cómo se complementan entre sí.

## Objetivos de Aprendizaje

Al finalizar este notebook, serás capaz de:

1. Comprender y aplicar el principio DRY para eliminar duplicación
2. Identificar y evitar complejidad innecesaria siguiendo KISS
3. Reconocer cuándo aplicar cada principio y cuándo no
4. Balancear DRY y KISS para crear abstracciones apropiadas
5. Refactorizar código existente aplicando ambos principios

## Principio DRY: Don't Repeat Yourself

### 🎯 Contexto: Por Qué Importa

**Problema real en Data/IA**: 

Validación de features en 15 lugares del código. Cambias regla de validación (ej: edad máxima 120 → 150). **Actualizas 12 lugares, olvidas 3** 💥 Producción: 3 lugares rechazan datos válidos. Pipeline falla. 2 horas debugging para encontrar los 3 lugares olvidados.

Con DRY: validación en UNA función. Cambias en un lugar. **Todos los usos actualizados automáticamente** ✅ 5 minutos vs 2 horas.

**Ejemplo concreto para juniors**:

Cálculo de descuento copiado en 8 funciones. Bug: descuento 20% calculado como `price * 0.2` (incorrecto) en vez de `price * 0.8`. **Arreglas 7 lugares, olvidas 1** 💥 Ese lugar sigue dando precios incorrectos.

Con DRY: función `calculate_discount()`. Bug arreglado en un lugar. **Todos los usos corregidos** ✅

**Consecuencias de NO usarlo**:
- **Bugs multiplicados** → mismo bug en 10 lugares
- **Cambios peligrosos** → fácil olvidar actualizar alguna copia
- **Inconsistencias** → copias divergen con el tiempo
- **Mantenimiento 10x más caro** → cada cambio requiere actualizar N lugares
- **Testing imposible** → tienes que testear 10 copias del mismo código

### 📚 El Concepto

**Definición técnica**:

**DRY** (Don't Repeat Yourself): Cada pieza de conocimiento debe tener **una representación única** en el sistema. Duplicación = violación de DRY.

**Cómo funciona internamente**:
1. Identificas código/lógica duplicada
2. Extraes a función/clase/constante
3. Reemplazas todas las copias con llamadas a la versión única
4. Ahora cambios se hacen en UN solo lugar
5. Todos los usos se benefician automáticamente

**Terminología clave**:
- **Duplication**: Mismo código/lógica en múltiples lugares
- **Single Source of Truth**: Una sola fuente autoritativa
- **Abstraction**: Extraer concepto común
- **WET**: Write Everything Twice (antipatrón, opuesto a DRY)

**Tipos de duplicación**:
```python
# 1. Duplicación literal (copy-paste)
if age < 0 or age > 150:  # Aparece en 5 lugares
    raise ValueError("Invalid age")

# 2. Duplicación estructural (misma estructura, diferentes valores)
if user.role == "admin":
    # ...
if user.role == "moderator":
    # ...
# Mejor: strategy pattern o dict lookup

# 3. Duplicación de conocimiento (misma regla de negocio)
# Archivo A: MAX_AGE = 150
# Archivo B: if age > 150  # Duplica el conocimiento
```

### ❌ Ejemplo Incorrecto: Validación Duplicada

In [ ]:
# Bad: Duplicated validation logic
def create_user(name: str, email: str, age: int) -> dict:
    """
    Create a new user.
    
    :param name: User's name
    :type name: str
    :param email: User's email
    :type email: str
    :param age: User's age
    :type age: int
    :return: User dictionary
    :rtype: dict
    :raises ValueError: If validation fails
    """
    # Validation logic
    if not name or len(name) < 2:
        raise ValueError("Name must be at least 2 characters")
    if not email or "@" not in email:
        raise ValueError("Invalid email address")
    if age < 0 or age > 150:
        raise ValueError("Invalid age")
    
    return {"name": name, "email": email, "age": age}

def update_user(user_id: int, name: str, email: str, age: int) -> dict:
    """
    Update an existing user.
    
    :param user_id: User ID
    :type user_id: int
    :param name: User's name
    :type name: str
    :param email: User's email
    :type email: str
    :param age: User's age
    :type age: int
    :return: Updated user dictionary
    :rtype: dict
    :raises ValueError: If validation fails
    """
    # Same validation logic duplicated!
    if not name or len(name) < 2:
        raise ValueError("Name must be at least 2 characters")
    if not email or "@" not in email:
        raise ValueError("Invalid email address")
    if age < 0 or age > 150:
        raise ValueError("Invalid age")
    
    return {"id": user_id, "name": name, "email": email, "age": age}

# Test the functions
try:
    user = create_user("Alice", "alice@example.com", 30)
    print(f"Created: {user}")
except ValueError as e:
    print(f"Error: {e}")

### ✅ Ejemplo Correcto: Validación Centralizada (DRY)

In [ ]:
# Good: DRY - Extract validation functions
def validate_name(name: str) -> None:
    """
    Validate user name.
    
    :param name: User's name
    :type name: str
    :raises ValueError: If name is invalid
    """
    if not name or len(name) < 2:
        raise ValueError("Name must be at least 2 characters")

def validate_email(email: str) -> None:
    """
    Validate email address.
    
    :param email: User's email
    :type email: str
    :raises ValueError: If email is invalid
    """
    if not email or "@" not in email:
        raise ValueError("Invalid email address")

def validate_age(age: int) -> None:
    """
    Validate user age.
    
    :param age: User's age
    :type age: int
    :raises ValueError: If age is invalid
    """
    if age < 0 or age > 150:
        raise ValueError("Invalid age")

def validate_user_data(name: str, email: str, age: int) -> None:
    """
    Validate all user data.
    
    :param name: User's name
    :type name: str
    :param email: User's email
    :type email: str
    :param age: User's age
    :type age: int
    :raises ValueError: If validation fails
    """
    validate_name(name)
    validate_email(email)
    validate_age(age)

def create_user(name: str, email: str, age: int) -> dict:
    """
    Create a new user.
    
    :param name: User's name
    :type name: str
    :param email: User's email
    :type email: str
    :param age: User's age
    :type age: int
    :return: User dictionary
    :rtype: dict
    """
    validate_user_data(name, email, age)
    return {"name": name, "email": email, "age": age}

def update_user(user_id: int, name: str, email: str, age: int) -> dict:
    """
    Update an existing user.
    
    :param user_id: User ID
    :type user_id: int
    :param name: User's name
    :type name: str
    :param email: User's email
    :type email: str
    :param age: User's age
    :type age: int
    :return: Updated user dictionary
    :rtype: dict
    """
    validate_user_data(name, email, age)
    return {"id": user_id, "name": name, "email": email, "age": age}

# Test the refactored functions
user = create_user("Alice", "alice@example.com", 30)
print(f"Created: {user}")

updated = update_user(1, "Alice Smith", "alice.smith@example.com", 31)
print(f"Updated: {updated}")

### 💡 Aprendizaje Clave - DRY

**Puntos críticos a recordar**:
1. **Una sola fuente de verdad** → cada concepto en UN solo lugar
2. **Extrae duplicación** → si copias código, extrae a función
3. **Cambios en un lugar** → modificación se propaga automáticamente
4. **Testing más fácil** → testeas función una vez, no N copias
5. **Bugs no se multiplican** → arreglas en un lugar, todos los usos corregidos

**Cómo desarrollar intuición**:
- **Pregúntate**: "¿He escrito este código antes?"
  - SÍ → extrae a función/constante ✅
  - NO → está bien dejarlo inline (por ahora) ✅

- **Pregúntate**: "¿Si cambio esta regla, cuántos lugares debo actualizar?"
  - Más de 1 → violación de DRY, refactoriza ❌
  - Solo 1 → perfecto ✅

- **Pregúntate**: "¿Estoy copiando y pegando?"
  - SÍ → STOP! Extrae primero ❌
  - NO → continúa ✅

**Cuándo usar / NO usar**:
- ✅ **Aplicar DRY cuando**:
  - Mismo código en 2+ lugares
  - Misma regla de negocio expresada múltiples veces
  - Misma validación repetida
  - Mismos cálculos duplicados
  - Mismas constantes hardcodeadas
- ❌ **NO aplicar DRY cuando**:
  - Código similar pero conceptualmente diferente (coincidencia)
  - Abstracción sería más compleja que duplicación
  - Solo aparece 1 vez (no hay duplicación aún)
  - Dominios diferentes (ej: validación user vs validación product)

**Regla de 3**:
```python
# Primera vez: escribe inline
if age < 0 or age > 150:
    raise ValueError("Invalid age")

# Segunda vez: nota la duplicación, pero espera
if age < 0 or age > 150:  # Hmm, otra vez...
    raise ValueError("Invalid age")

# Tercera vez: REFACTORIZA
def validate_age(age: int) -> None:
    if age < 0 or age > 150:
        raise ValueError("Invalid age")
```

**Niveles de abstracción DRY**:
1. **Constantes**: `MAX_AGE = 150` (no hardcodear)
2. **Funciones**: `validate_age()` (lógica repetida)
3. **Clases**: `Validator` (múltiples validaciones relacionadas)
4. **Módulos**: `validators.py` (conjunto de validadores)

**Referencia oficial:** [The Pragmatic Programmer - DRY Principle](https://pragprog.com/titles/tpp20/the-pragmatic-programmer-20th-anniversary-edition/)